# Analysis of the Film Industry for Business Expansion
##  Project Overview

In response to the growing trend of major corporations producing original content, our company is launching a data-driven initiative to establish a successful movie studio. Given our limited experience in film production, this project relies on a comprehensive analysis of film industry datasets to extract actionable insights.

Our primary goal is to understand which aspects of the film industry—genres, directors, languages, and markets—are most closely associated with critical acclaim and financial success. These insights will inform a strategy that aligns creative projects with business objectives.


##  Business Objective

The film production landscape is increasingly influenced by streaming platforms, international markets, and digital distribution. To navigate this complexity, we must identify the key factors behind successful films using reliable, data-backed evidence.

Through this analysis, we aim to:

- Identify **genres** consistently praised by critics.
- Determine **genres** with the highest **Return on Investment (ROI)** potential.
- Recognize **directors** whose films perform well in **international markets**.
- Discover **languages** most commonly associated with successful films.


## Strategic Impact

By answering these core questions, we will equip our studio team with the knowledge to:

- Prioritize projects that are both critically and commercially promising.
- Invest wisely in film concepts with high audience appeal and strong market potential.
- Build a foundation for long-term success in the competitive entertainment industry.

Let’s begin by exploring the data and uncovering the stories behind the numbers.

# Loading and Exploring Datasets

Before diving into analysis, it is essential to thoroughly explore each of the datasets and database tables we are working with. Our data sources span multiple reputable platforms, providing a comprehensive view of the film industry from both commercial and critical perspectives.

### Datasets in Use:

- **Box Office Mojo**
- **IMDb**
- **Rotten Tomatoes (Movies)**
- **Rotten Tomatoes (Critic Reviews)**
- **TheMovieDB**
- **The Numbers**
- **im.db** (SQLite database)


###  Why Data Exploration Matters

Data exploration is a critical step in any data-driven project. It helps us:

- Understand the **structure, quality, and characteristics** of each dataset.
- Identify and address issues such as **missing values**, **duplicates**, and **inconsistent formatting**.
- Analyze **distributions** and **relationships** between variables to uncover initial patterns.
- Develop an informed plan for **data cleaning**, **transformation**, and **integration**.

By doing this, we reduce the risk of errors.

In [27]:
# import the necessary libraries
import itertools
import numpy as np
import pandas as pd 
from numbers import Number
import sqlite3
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from matplotlib.patches import Rectangle
from matplotlib.gridspec import GridSpec
import matplotlib.patches as mpatches
from matplotlib.backends.backend_pdf import PdfPages
import io
import base64
warnings.filterwarnings('ignore')

import pickle
import os
print(os.getcwd())

plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)





c:\Users\USER\Documents\moringaprojects\phase 2 project\Phase-G-2-Project


Load all datasets into pandas dataframes

In [19]:
bom_df = pd.read_csv("bom.movie_gross.csv")
bom_df.head()


,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000,2010
3,Inception,WB,292600000.0,535700000,2010
4,Shrek Forever After,P/DW,238700000.0,513900000,2010


In [30]:
rt_reviews_df = pd.read_csv("rt.reviews.tsv", sep='\t',encoding='latin1')
rt_reviews_df.head()

,id,review,rating,fresh,critic,top_critic,publisher,date
0,3,A distinctly gallows take on contemporary fina...,3/5,fresh,PJ Nabarro,0,Patrick Nabarro,"November 10, 2018"
1,3,It's an allegory in search of a meaning that n...,NaN,rotten,Annalee Newitz,0,io9.com,"May 23, 2018"
2,3,... life lived in a bubble in financial dealin...,NaN,fresh,Sean Axmaker,0,Stream on Demand,"January 4, 2018"
3,3,Continuing along a line introduced in last yea...,NaN,fresh,Daniel Kasman,0,MUBI,"November 16, 2017"
4,3,... a perverse twist on neorealism...,NaN,fresh,NaN,0,Cinema Scope,"October 12, 2017"


In [31]:
tmdb_df = pd.read_csv('tmdb.movies.csv')
tmdb_df.head()

,Unnamed: 0,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count
0,0,"[12, 14, 10751]",12444,en,Harry Potter and the Deathly Hallows: Part 1,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788
1,1,"[14, 12, 16, 10751]",10191,en,How to Train Your Dragon,28.734,2010-03-26,How to Train Your Dragon,7.7,7610
2,2,"[12, 28, 878]",10138,en,Iron Man 2,28.515,2010-05-07,Iron Man 2,6.8,12368
3,3,"[16, 35, 10751]",862,en,Toy Story,28.005,1995-11-22,Toy Story,7.9,10174
4,4,"[28, 878, 12]",27205,en,Inception,27.920,2010-07-16,Inception,8.3,22186


In [32]:
tn_budget_df = pd.read_csv('tn.movie_budgets.csv')
tn_budget_df

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"
...,...,...,...,...,...,...
5777,78,"Dec 31, 2018",Red 11,"$7,000",$0,$0
5778,79,"Apr 2, 1999",Following,"$6,000","$48,482","$240,495"
5779,80,"Jul 13, 2005",Return to the Land of Wonders,"$5,000","$1,338","$1,338"
5780,81,"Sep 29, 2015",A Plague So Pleasant,"$1,400",$0,$0


In [33]:
rt_info_df = pd.read_csv('rt.movie_info.tsv', sep='\t')
rt_info_df

,id,synopsis,rating,genre,director,writer,theater_date,dvd_date,currency,box_office,runtime,studio
0,1,"This gritty, fast-paced, and innovative police...",R,Action and Adventure|Classics|Drama,William Friedkin,Ernest Tidyman,"Oct 9, 1971","Sep 25, 2001",NaN,NaN,104 minutes,NaN
1,3,"New York City, not-too-distant-future: Eric Pa...",R,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,"Aug 17, 2012","Jan 1, 2013",$,"600,000",108 minutes,Entertainment One
2,5,Illeana Douglas delivers a superb performance ...,R,Drama|Musical and Performing Arts,Allison Anders,Allison Anders,"Sep 13, 1996","Apr 18, 2000",NaN,NaN,116 minutes,NaN
3,6,Michael Douglas runs afoul of a treacherous su...,R,Drama|Mystery and Suspense,Barry Levinson,Paul Attanasio|Michael Crichton,"Dec 9, 1994","Aug 27, 1997",NaN,NaN,128 minutes,NaN
4,7,NaN,NR,Drama|Romance,Rodney Bennett,Giles Cooper,NaN,NaN,NaN,NaN,200 minutes,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1555,1996,Forget terrorists or hijackers -- there's a ha...,R,Action and Adventure|Horror|Mystery and Suspense,NaN,NaN,"Aug 18, 2006","Jan 2, 2007",$,"33,886,034",106 minutes,New Line Cinema
1556,1997,The popular Saturday Night Live sketch was exp...,PG,Comedy|Science Fiction and Fantasy,Steve Barron,Terry Turner|Tom Davis|Dan Aykroyd|Bonnie Turner,"Jul 23, 1993","Apr 17, 2001",NaN,NaN,88 minutes,Paramount Vantage
1557,1998,"Based on a novel by Richard Powell, when the l...",G,Classics|Comedy|Drama|Musical and Performing Arts,Gordon Douglas,NaN,"Jan 1, 1962","May 11, 2004",NaN,NaN,111 minutes,NaN
1558,1999,The Sandlot is a coming-of-age story about a g...,PG,Comedy|Drama|Kids and Family|Sports and Fitness,David Mickey Evans,David Mickey Evans|Robert Gunter,"Apr 1, 1993","Jan 29, 2002",NaN,NaN,101 minutes,NaN


In [ ]:
# establish a connection to the database
path = ("Data\im.db\im.db")
conn = sqlite3.connect(path)

In [ ]:
#create a cursor 
cursor = conn.cursor()

In [37]:
# Query the SQLite database to list all table names
pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)

,name
0,movie_basics
1,directors
2,known_for
3,movie_akas
4,movie_ratings
5,persons
6,principals
7,writers


# Data Preparation & Cleaning

Before conducting meaningful analysis, it is essential to prepare and clean the data appropriately. Although our earlier exploration suggests that the datasets are relatively high quality, they originate from diverse sources and therefore require alignment for consistency and accuracy.

Data preparation involves:

- Filtering relevant columns
- Renaming fields for consistency
- Handling duplicates or inconsistencies
- Parsing multi-value fields
- Merging datasets where necessary

This step ensures that our analysis is based on a reliable, well-structured foundation.



## Preparing Rotten Tomatoes Ratings and Genres Data

To explore the relationship between **movie genres**, **ratings**, and **success metrics**, we focus first on preparing the **Rotten Tomatoes Movies** dataset. This dataset offers rich metadata for a wide array of films, including elements crucial to understanding **audience perception** and **genre trends**.

Our objectives here are to:

- Extract key attributes that reflect critical and audience reception
- Standardize genre information for analytical use
- Enable cross-comparison with financial and market performance data



## Extracting Ratings and Genres

We will extract and clean the following key columns from the dataset:

- **`movie_title`** – The title of the movie as listed on Rotten Tomatoes.
- **`genres`** – A comma-separated list of genres assigned to each movie (e.g., *Drama, Comedy, Action*).
- **`audience_rating`** – The average rating given by audiences, usually on a scale of 0 to 100.
- **`tomatometer_rating`** – The percentage of positive critic reviews (Tomatometer score).

> **Note**: Since many movies belong to **multiple genres**, genre parsing must be handled carefully to allow accurate aggregation and trend analysis.

By preparing this data effectively, we enable deeper insights into how **genre** and **reception** correlate with **film success**, supporting the strategic decision-making goals of our new studio.

In [46]:
#using sql
genre = pd.read_sql_query("""SELECT genres FROM movie_basics JOIN movie_ratings USING (movie_id) """, conn)
genre

,genres
0,"Action,Crime,Drama"
1,"Biography,Drama"
2,Drama
3,"Comedy,Drama"
4,"Comedy,Drama,Fantasy"
...,...
73851,Documentary
73852,"Drama,Family"
73853,Documentary
73854,None
